In [ ]:
# %% [markdown]
# # 🎯 Argument Mining para Transcripciones Estudiantiles - Fase 1
# ## Prueba Inicial con Texto Real
#
# Este notebook te permite probar el análisis de argumentación en tu transcripción.
#
# **Configuración:** XLM-RoBERTa + Clasificación CLAIM/EVIDENCE/NONE

# %% [markdown]
# ## 1️⃣ Instalación y Setup

# %%
# Instalar librerías necesarias
!pip install transformers torch sentencepiece sacremoses -q
!pip install spacy -q
!python -m spacy download es_core_news_sm -q

# Librerías para visualización
!pip install plotly -q

print("✅ Librerías instaladas")

# %%
# Imports necesarios
import torch
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaForSequenceClassification,
    pipeline
)
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
import re
import json
from IPython.display import HTML, display
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Verificar GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Usando: {device}")
print(f"🔋 GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# %% [markdown]
# ## 2️⃣ Preparación del Modelo
#
# Usaremos un modelo preentrenado para empezar rápidamente.

# %%
class ArgumentMiner:
    def __init__(self):
        """Inicializa el modelo para minería de argumentación"""
        print("🔄 Cargando modelo XLM-RoBERTa...")

        # Usaremos un modelo preentrenado en clasificación de secuencias
        # Para la prueba inicial, adaptaremos uno existente
        self.model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

        # Cargar tokenizador y modelo
        self.tokenizer = XLMRobertaTokenizer.from_pretrained(self.model_name)
        self.model = XLMRobertaForSequenceClassification.from_pretrained(self.model_name)

        # Mover a GPU si está disponible
        self.model.to(device)
        self.model.eval()

        # Mapeo de etiquetas (adaptaremos el modelo de sentimiento)
        # Para Fase 1: NEGATIVE->NONE, NEUTRAL->EVIDENCE, POSITIVE->CLAIM
        self.label_map = {
            0: "NONE",
            1: "EVIDENCE",
            2: "CLAIM"
        }

        print("✅ Modelo cargado y listo")

    def preprocess_text(self, text: str) -> Dict:
        """Preprocesa la transcripción"""

        # Limpiar texto
        text = text.strip()

        # Marcar muletillas comunes
        muletillas = ["eh", "ehh", "este", "esto", "mm", "mmm", "ah", "ahh"]
        for muletilla in muletillas:
            text = re.sub(f"\\b{muletilla}\\b", f"[MULETILLA]", text, flags=re.IGNORECASE)

        # Marcar pausas
        text = re.sub(r'\.\.\.|…', '[PAUSA]', text)

        # Detectar posibles cambios de tema
        marcadores_cambio = ["ahora bien", "por otro lado", "en otro orden", "cambiando de tema"]
        segments = []
        current_segment = []
        sentences = text.split('.')

        for sent in sentences:
            sent = sent.strip()
            if not sent:
                continue

            # Verificar si hay marcador de cambio
            is_new_segment = any(marker in sent.lower() for marker in marcadores_cambio)

            if is_new_segment and current_segment:
                segments.append('. '.join(current_segment))
                current_segment = [sent]
            else:
                current_segment.append(sent)

        if current_segment:
            segments.append('. '.join(current_segment))

        return {
            'original': text,
            'cleaned': text,
            'segments': segments if segments else [text],
            'sentence_count': len(sentences)
        }

    def segment_into_clauses(self, text: str) -> List[str]:
        """Segmenta el texto en cláusulas"""

        # Marcadores de cláusulas
        clause_markers = [
            ', porque', ', ya que', ', debido a', ', por lo tanto',
            ', entonces', ', así que', ', pero', ', sin embargo',
            ', aunque', ', mientras', ', cuando', '. ', '? ', '! '
        ]

        clauses = []
        current = text

        # Dividir por marcadores manteniendo el marcador
        for marker in clause_markers:
            parts = current.split(marker)
            if len(parts) > 1:
                for i, part in enumerate(parts[:-1]):
                    if part.strip():
                        clauses.append(part.strip())
                current = parts[-1]

        if current.strip():
            clauses.append(current.strip())

        # Si no hay cláusulas, dividir por oraciones
        if not clauses or len(clauses) == 1:
            clauses = [s.strip() for s in re.split('[.!?]', text) if s.strip()]

        return clauses if clauses else [text]

    def classify_clause(self, clause: str) -> Tuple[str, float]:
        """Clasifica una cláusula como CLAIM, EVIDENCE o NONE"""

        # Tokenizar
        inputs = self.tokenizer(
            clause,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        ).to(device)

        # Predecir
        with torch.no_grad():
            outputs = self.model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            confidence = torch.max(predictions).item()
            predicted_class = torch.argmax(predictions, dim=-1).item()

        # Aplicar heurísticas para mejorar clasificación
        label = self.apply_heuristics(clause, self.label_map[predicted_class], confidence)

        return label, confidence

    def apply_heuristics(self, text: str, predicted_label: str, confidence: float) -> str:
        """Aplica reglas heurísticas para mejorar la clasificación"""

        text_lower = text.lower()

        # Patrones para CLAIM
        claim_patterns = [
            "creo que", "pienso que", "considero que", "mi opinión",
            "sostengo que", "argumento que", "propongo", "concluyo",
            "es importante", "es necesario", "debemos", "deberíamos",
            "¿acaso no", "¿no es cierto que"  # Preguntas retóricas
        ]

        # Patrones para EVIDENCE
        evidence_patterns = [
            "según", "de acuerdo con", "estudios muestran", "investigaciones",
            "por ejemplo", "como muestra", "datos indican", "estadísticas",
            "el autor dice", "cita:", "%", "número", "cifra"
        ]

        # Si hay alta confianza en un patrón, override
        if any(pattern in text_lower for pattern in claim_patterns):
            if confidence < 0.7 or predicted_label != "CLAIM":
                return "CLAIM"

        if any(pattern in text_lower for pattern in evidence_patterns):
            if confidence < 0.7 or predicted_label != "EVIDENCE":
                return "EVIDENCE"

        # Preguntas retóricas son CLAIM
        if text.strip().startswith("¿") and text.strip().endswith("?"):
            if "acaso" in text_lower or "no es" in text_lower:
                return "CLAIM"

        return predicted_label

    def analyze_text(self, text: str) -> Dict:
        """Análisis completo del texto"""

        print("🔍 Analizando texto...")

        # Preprocesar
        preprocessed = self.preprocess_text(text)

        # Analizar cada segmento
        results = []
        for segment_idx, segment in enumerate(preprocessed['segments']):
            print(f"   Segmento {segment_idx + 1}/{len(preprocessed['segments'])}")

            # Dividir en cláusulas
            clauses = self.segment_into_clauses(segment)

            # Clasificar cada cláusula
            segment_results = []
            for clause in clauses:
                if len(clause.strip()) < 10:  # Ignorar cláusulas muy cortas
                    continue

                label, confidence = self.classify_clause(clause)
                segment_results.append({
                    'text': clause,
                    'label': label,
                    'confidence': confidence,
                    'segment': segment_idx
                })

            results.extend(segment_results)

        # Calcular métricas
        metrics = self.calculate_metrics(results)

        print("✅ Análisis completado")

        return {
            'preprocessed': preprocessed,
            'results': results,
            'metrics': metrics
        }

    def calculate_metrics(self, results: List[Dict]) -> Dict:
        """Calcula métricas pedagógicas básicas"""

        if not results:
            return {}

        # Contar etiquetas
        label_counts = {"CLAIM": 0, "EVIDENCE": 0, "NONE": 0}
        for r in results:
            label_counts[r['label']] += 1

        total = len(results)

        # Métricas básicas
        metrics = {
            'total_clauses': total,
            'claim_count': label_counts['CLAIM'],
            'evidence_count': label_counts['EVIDENCE'],
            'none_count': label_counts['NONE'],
            'claim_percentage': (label_counts['CLAIM'] / total * 100) if total > 0 else 0,
            'evidence_percentage': (label_counts['EVIDENCE'] / total * 100) if total > 0 else 0,
            'claim_evidence_ratio': (label_counts['CLAIM'] / label_counts['EVIDENCE']) if label_counts['EVIDENCE'] > 0 else 0,
            'has_thesis': label_counts['CLAIM'] > 0,
            'has_evidence': label_counts['EVIDENCE'] > 0,
            'balance_score': min(label_counts['CLAIM'], label_counts['EVIDENCE']) / max(label_counts['CLAIM'], label_counts['EVIDENCE']) if max(label_counts['CLAIM'], label_counts['EVIDENCE']) > 0 else 0
        }

        # Evaluación pedagógica
        if metrics['claim_evidence_ratio'] > 2:
            metrics['feedback'] = "⚠️ Muchas afirmaciones sin suficiente evidencia"
        elif metrics['claim_evidence_ratio'] < 0.5:
            metrics['feedback'] = "⚠️ Mucha evidencia pero pocas afirmaciones claras"
        elif metrics['balance_score'] > 0.6:
            metrics['feedback'] = "✅ Buen balance entre afirmaciones y evidencia"
        else:
            metrics['feedback'] = "💡 Intenta balancear mejor tus argumentos con evidencia"

        return metrics

# Inicializar el modelo
miner = ArgumentMiner()

# %% [markdown]
# ## 3️⃣ Funciones de Visualización

# %%
def generate_html_highlights(results: List[Dict]) -> str:
    """Genera HTML con el texto anotado"""

    # Colores para cada etiqueta
    colors = {
        'CLAIM': '#e74c3c',      # Rojo
        'EVIDENCE': '#3498db',    # Azul
        'NONE': '#95a5a6'         # Gris
    }

    # CSS para el estilo
    html = """
    <style>
        .analysis-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            line-height: 1.8;
            max-width: 800px;
            margin: 20px auto;
            padding: 20px;
            background: white;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        .clause {
            padding: 2px 6px;
            margin: 2px;
            border-radius: 4px;
            display: inline;
            transition: all 0.3s ease;
        }
        .clause:hover {
            transform: scale(1.05);
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
        }
        .claim {
            background-color: rgba(231, 76, 60, 0.2);
            border-bottom: 3px solid #e74c3c;
        }
        .evidence {
            background-color: rgba(52, 152, 219, 0.2);
            border-bottom: 3px solid #3498db;
        }
        .none {
            background-color: rgba(149, 165, 166, 0.1);
            color: #7f8c8d;
        }
        .legend {
            margin-bottom: 20px;
            padding: 10px;
            background: #f8f9fa;
            border-radius: 5px;
        }
        .legend-item {
            display: inline-block;
            margin-right: 20px;
        }
        .legend-color {
            display: inline-block;
            width: 20px;
            height: 20px;
            margin-right: 5px;
            border-radius: 3px;
            vertical-align: middle;
        }
        .tooltip {
            position: relative;
            display: inline-block;
        }
        .tooltip .tooltiptext {
            visibility: hidden;
            background-color: #333;
            color: #fff;
            text-align: center;
            padding: 5px 10px;
            border-radius: 6px;
            position: absolute;
            z-index: 1;
            bottom: 125%;
            left: 50%;
            margin-left: -60px;
            opacity: 0;
            transition: opacity 0.3s;
            font-size: 12px;
            white-space: nowrap;
        }
        .tooltip:hover .tooltiptext {
            visibility: visible;
            opacity: 1;
        }
    </style>
    """

    # Leyenda
    html += """
    <div class="analysis-container">
        <h3>📝 Texto Analizado</h3>
        <div class="legend">
            <span class="legend-item">
                <span class="legend-color" style="background-color: #e74c3c;"></span>
                <strong>CLAIM</strong> (Afirmaciones)
            </span>
            <span class="legend-item">
                <span class="legend-color" style="background-color: #3498db;"></span>
                <strong>EVIDENCE</strong> (Evidencia)
            </span>
            <span class="legend-item">
                <span class="legend-color" style="background-color: #95a5a6;"></span>
                <strong>NONE</strong> (Otro)
            </span>
        </div>
        <div class="text-content">
    """

    # Añadir cada cláusula con su clasificación
    for i, result in enumerate(results):
        label = result['label'].lower()
        confidence = result['confidence']
        text = result['text']

        html += f'''
        <span class="clause {label} tooltip">
            {text}
            <span class="tooltiptext">{result['label']} ({confidence:.1%})</span>
        </span>
        '''

        # Añadir punto si no lo tiene
        if not text.endswith(('.', '!', '?')):
            html += '. '
        else:
            html += ' '

    html += """
        </div>
    </div>
    """

    return html

def generate_metrics_dashboard(metrics: Dict) -> str:
    """Genera un dashboard con las métricas"""

    html = """
    <style>
        .metrics-dashboard {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 800px;
            margin: 20px auto;
            padding: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-radius: 15px;
            color: white;
        }
        .metrics-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin: 20px 0;
        }
        .metric-card {
            background: rgba(255, 255, 255, 0.2);
            padding: 15px;
            border-radius: 10px;
            text-align: center;
            backdrop-filter: blur(10px);
        }
        .metric-value {
            font-size: 2em;
            font-weight: bold;
            margin: 5px 0;
        }
        .metric-label {
            font-size: 0.9em;
            opacity: 0.9;
        }
        .feedback-box {
            background: rgba(255, 255, 255, 0.2);
            padding: 15px;
            border-radius: 10px;
            margin-top: 20px;
            text-align: center;
            font-size: 1.1em;
        }
    </style>
    """

    html += f"""
    <div class="metrics-dashboard">
        <h2>📊 Análisis de tu Argumentación</h2>

        <div class="metrics-grid">
            <div class="metric-card">
                <div class="metric-value">{metrics.get('claim_count', 0)}</div>
                <div class="metric-label">Afirmaciones (Claims)</div>
            </div>

            <div class="metric-card">
                <div class="metric-value">{metrics.get('evidence_count', 0)}</div>
                <div class="metric-label">Evidencias</div>
            </div>

            <div class="metric-card">
                <div class="metric-value">{metrics.get('claim_percentage', 0):.0f}%</div>
                <div class="metric-label">Porcentaje Claims</div>
            </div>

            <div class="metric-card">
                <div class="metric-value">{metrics.get('evidence_percentage', 0):.0f}%</div>
                <div class="metric-label">Porcentaje Evidencia</div>
            </div>

            <div class="metric-card">
                <div class="metric-value">{metrics.get('balance_score', 0):.2f}</div>
                <div class="metric-label">Balance (0-1)</div>
            </div>

            <div class="metric-card">
                <div class="metric-value">{"✅" if metrics.get('has_thesis') else "❌"}</div>
                <div class="metric-label">Tesis Detectada</div>
            </div>
        </div>

        <div class="feedback-box">
            <strong>Retroalimentación:</strong><br>
            {metrics.get('feedback', 'Analizando...')}
        </div>
    </div>
    """

    return html

def create_interactive_graph(results: List[Dict]) -> go.Figure:
    """Crea un grafo interactivo de la estructura argumentativa"""

    # Preparar datos para el grafo
    nodes = []
    edges = []

    # Crear nodo raíz
    nodes.append({
        'id': 0,
        'label': 'Presentación',
        'type': 'root',
        'text': 'Análisis Completo'
    })

    # Agrupar por tipo
    claims = [r for r in results if r['label'] == 'CLAIM']
    evidences = [r for r in results if r['label'] == 'EVIDENCE']

    # Añadir nodos de claims
    claim_start_id = 1
    for i, claim in enumerate(claims):
        node_id = claim_start_id + i
        nodes.append({
            'id': node_id,
            'label': f'Claim {i+1}',
            'type': 'claim',
            'text': claim['text'][:50] + '...' if len(claim['text']) > 50 else claim['text']
        })
        edges.append((0, node_id))

    # Añadir nodos de evidence
    evidence_start_id = claim_start_id + len(claims)
    for i, evidence in enumerate(evidences):
        node_id = evidence_start_id + i
        nodes.append({
            'id': node_id,
            'label': f'Evidence {i+1}',
            'type': 'evidence',
            'text': evidence['text'][:50] + '...' if len(evidence['text']) > 50 else evidence['text']
        })

        # Conectar evidencia al claim más cercano (simplificado)
        if claims:
            nearest_claim = claim_start_id + (i % len(claims))
            edges.append((nearest_claim, node_id))

    # Crear visualización con Plotly
    edge_trace = []
    for edge in edges:
        x0, y0 = nodes[edge[0]]['id'] * 2, nodes[edge[0]]['id']
        x1, y1 = nodes[edge[1]]['id'] * 2, nodes[edge[1]]['id'] * 1.5
        edge_trace.append(go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode='lines',
            line=dict(width=0.5, color='#888'),
            hoverinfo='none'
        ))

    node_trace = go.Scatter(
        x=[node['id'] * 2 for node in nodes],
        y=[node['id'] * (1.5 if node['type'] != 'root' else 1) for node in nodes],
        mode='markers+text',
        text=[node['label'] for node in nodes],
        textposition="top center",
        hovertext=[node['text'] for node in nodes],
        hoverinfo='text',
        marker=dict(
            showscale=False,
            colorscale='Viridis',
            size=15,
            color=[
                '#e74c3c' if node['type'] == 'claim' else
                '#3498db' if node['type'] == 'evidence' else
                '#2ecc71'
                for node in nodes
            ],
            line_width=2
        )
    )

    fig = go.Figure(data=edge_trace + [node_trace],
                   layout=go.Layout(
                       title='Estructura Argumentativa',
                       showlegend=False,
                       hovermode='closest',
                       margin=dict(b=20,l=5,r=5,t=40),
                       xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                       yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                       height=500
                   ))

    return fig

# %% [markdown]
# ## 4️⃣ CARGAR TU TRANSCRIPCIÓN
#
# **Ejecuta la siguiente celda y sube tu archivo cuando aparezca el botón**

# %%
# 📁 CARGAR ARCHIVO DE TRANSCRIPCIÓN
print("📤 Por favor, sube tu archivo de transcripción")
print("Formatos aceptados: .txt, .doc, .docx, .pdf")
print("-" * 50)

from google.colab import files
import io

# Subir archivo
uploaded = files.upload()

# Procesar el archivo subido
if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"\n✅ Archivo recibido: {filename}")

    # Detectar tipo de archivo y procesar
    if filename.endswith('.txt'):
        # Archivo de texto plano
        mi_transcripcion = uploaded[filename].decode('utf-8')
        print(f"📄 Texto cargado: {len(mi_transcripcion)} caracteres")

    elif filename.endswith('.docx'):
        # Archivo Word
        !pip install python-docx -q
        from docx import Document
        import io

        doc = Document(io.BytesIO(uploaded[filename]))
        mi_transcripcion = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        print(f"📄 Documento Word procesado: {len(mi_transcripcion)} caracteres")

    elif filename.endswith('.pdf'):
        # Archivo PDF
        !pip install PyPDF2 -q
        import PyPDF2
        import io

        pdf_reader = PyPDF2.PdfReader(io.BytesIO(uploaded[filename]))
        mi_transcripcion = ''
        for page in pdf_reader.pages:
            mi_transcripcion += page.extract_text()
        print(f"📄 PDF procesado: {len(mi_transcripcion)} caracteres")

    else:
        # Intentar leer como texto plano
        try:
            mi_transcripcion = uploaded[filename].decode('utf-8')
            print(f"📄 Archivo procesado como texto: {len(mi_transcripcion)} caracteres")
        except:
            print("❌ Error: No se pudo leer el archivo. Por favor usa formato .txt, .docx o .pdf")
            mi_transcripcion = ""

    # Mostrar preview del texto
    if mi_transcripcion:
        print("\n📝 PREVIEW DEL TEXTO (primeros 500 caracteres):")
        print("-" * 50)
        print(mi_transcripcion[:500])
        if len(mi_transcripcion) > 500:
            print("\n... [texto continúa]")
        print("-" * 50)
        print(f"\n✅ Listo para analizar {len(mi_transcripcion.split())} palabras")

else:
    print("❌ No se subió ningún archivo")
    print("\n💡 ALTERNATIVA: Pega tu texto directamente aquí:")
    mi_transcripcion = """
    Pega aquí tu transcripción si prefieres no subir archivo.
    """

# %% [markdown]
# ## 5️⃣ Ejecutar Análisis

# %%
# Analizar el texto
print("🚀 Iniciando análisis de argumentación...")
print("="*50)

# Ejecutar análisis
analysis = miner.analyze_text(mi_transcripcion)

print("\n📊 RESULTADOS DEL ANÁLISIS")
print("="*50)

# Mostrar métricas
metrics = analysis['metrics']
print(f"\n✅ Tesis detectada: {'Sí' if metrics['has_thesis'] else 'No'}")
print(f"📝 Total de cláusulas analizadas: {metrics['total_clauses']}")
print(f"🔴 Claims (Afirmaciones): {metrics['claim_count']} ({metrics['claim_percentage']:.1f}%)")
print(f"🔵 Evidence (Evidencia): {metrics['evidence_count']} ({metrics['evidence_percentage']:.1f}%)")
print(f"⚪ Otro contenido: {metrics['none_count']}")
print(f"\n💡 {metrics['feedback']}")

# %% [markdown]
# ## 6️⃣ Visualización de Resultados

# %%
# Generar y mostrar HTML con highlights
html_highlights = generate_html_highlights(analysis['results'])
display(HTML(html_highlights))

# %%
# Mostrar dashboard de métricas
metrics_html = generate_metrics_dashboard(analysis['metrics'])
display(HTML(metrics_html))

# %%
# Crear y mostrar grafo interactivo
if analysis['results']:
    fig = create_interactive_graph(analysis['results'])
    fig.show()
else:
    print("No hay suficientes datos para crear el grafo")

# %% [markdown]
# ## 7️⃣ Exportar Resultados

# %%
# Guardar resultados en JSON
output_filename = "analisis_argumentacion.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump({
        'metrics': analysis['metrics'],
        'results': analysis['results'],
        'text_segments': analysis['preprocessed']['segments']
    }, f, ensure_ascii=False, indent=2)

print(f"✅ Resultados guardados en: {output_filename}")

# Guardar HTML
html_output = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Análisis de Argumentación</title>
    <meta charset="utf-8">
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 20px;
            background: #f5f5f5;
        }}
        .container {{
            max-width: 1200px;
            margin: 0 auto;
        }}
    </style>
</head>
<body>
    <div class="container">
        {metrics_html}
        {html_highlights}
    </div>
</body>
</html>
"""

html_filename = "analisis_argumentacion.html"
with open(html_filename, 'w', encoding='utf-8') as f:
    f.write(html_output)

print(f"✅ HTML guardado en: {html_filename}")

# DESCARGAR ARCHIVOS AUTOMÁTICAMENTE
print("\n📥 DESCARGANDO ARCHIVOS A TU COMPUTADORA...")
from google.colab import files

# Descargar HTML
try:
    files.download(html_filename)
    print(f"✅ HTML descargado: {html_filename}")
except:
    print(f"⚠️ No se pudo descargar automáticamente. Busca {html_filename} en el panel de archivos")

# Descargar JSON
try:
    files.download(output_filename)
    print(f"✅ JSON descargado: {output_filename}")
except:
    print(f"⚠️ No se pudo descargar automáticamente. Busca {output_filename} en el panel de archivos")

print("\n💡 Si no se descargaron automáticamente:")
print("   1. Abre el panel de archivos (ícono 📁 en la izquierda)")
print("   2. Busca los archivos .html y .json")
print("   3. Click derecho → Download")

# %% [markdown]
# ## 8️⃣ Análisis Detallado por Cláusula

# %%
# Ver análisis detallado de cada cláusula
print("\n📋 ANÁLISIS DETALLADO POR CLÁUSULA")
print("="*50)

df_results = pd.DataFrame(analysis['results'])
if not df_results.empty:
    # Mostrar tabla con resultados
    df_display = df_results[['text', 'label', 'confidence']].copy()
    df_display['confidence'] = df_display['confidence'].apply(lambda x: f"{x:.1%}")
    df_display.columns = ['Texto', 'Clasificación', 'Confianza']

    # Mostrar primeras 10 para no saturar
    display(df_display.head(10))

    if len(df_display) > 10:
        print(f"\n... y {len(df_display) - 10} cláusulas más")
else:
    print("No hay resultados para mostrar")

# %% [markdown]
# ## 💡 Próximos Pasos
#
# ### Si los resultados son buenos:
# 1. **Fine-tuning**: Entrenar el modelo con tus propios datos anotados
# 2. **Más datos**: Añadir más transcripciones para mejorar
# 3. **Hugging Face**: Subir el modelo para compartirlo
#
# ### Si necesita mejoras:
# 1. **Ajustar heurísticas**: Modificar los patrones en `apply_heuristics()`
# 2. **Cambiar modelo base**: Probar con BERTO o otro modelo
# 3. **Anotar ejemplos**: Crear dataset con 20-30 ejemplos bien anotados
#
# ### Para producción:
# 1. **API**: Crear endpoint para procesar transcripciones automáticamente
# 2. **Interface**: Desarrollar interfaz web para profesores/estudiantes
# 3. **Integración**: Conectar con plataformas educativas

# %%
print("\n🎉 ¡Análisis completado!")
print("\n📝 INSTRUCCIONES:")
print("1. Pega tu transcripción en la celda marcada")
print("2. Ejecuta todas las celdas (Runtime > Run all)")
print("3. Revisa los resultados en las visualizaciones")
print("4. Descarga los archivos HTML y JSON generados")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 59.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ Librerías instaladas
🖥️ Usando: cuda
🔋 GPU disponible: True
   GPU: Tesla T4
🔄 Cargando modelo XLM-RoBERTa...
✅ Modelo cargado y listo
📤 Por favor, sube tu archivo de transcripción
Formatos aceptados: .txt, .doc, .docx, .pdf
--------------------------------------------------


Saving cambioclimatico.txt to cambioclimatico (1).txt

✅ Archivo recibido: cambioclimatico (1).txt
📄 Texto cargado: 11615 caracteres

📝 PREVIEW DEL TEXTO (primeros 500 caracteres):
--------------------------------------------------
TRANSCRIPCIÓN COMPLETA:

 El cambio climático no es en absoluto un debate, sino una realidad. Con esta premisa partiremos este vídeo de Ecología Verde en el que te explicaremos cinco argumentos frente a los negacionistas del cambio climático basados por supuesto en evidencias científicas. Vamos allá. ¿Cómo es posible que haya cambio climático si está haciendo frío? Empezaremos con un clásico argumento negacionista. La respuesta ante esta preg

... [texto continúa]
--------------------------------------------------

✅ Listo para analizar 1944 palabras
🚀 Iniciando análisis de argumentación...
🔍 Analizando texto...
   Segmento 1/1
✅ Análisis completado

📊 RESULTADOS DEL ANÁLISIS

✅ Tesis detectada: Sí
📝 Total de cláusulas analizadas: 23
🔴 Claims (Afirmaciones):

✅ Resultados guardados en: analisis_argumentacion.json
✅ HTML guardado en: analisis_argumentacion.html

📥 DESCARGANDO ARCHIVOS A TU COMPUTADORA...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ JSON descargado: analisis_argumentacion.json

💡 Si no se descargaron automáticamente:
   1. Abre el panel de archivos (ícono 📁 en la izquierda)
   2. Busca los archivos .html y .json
   3. Click derecho → Download

📋 ANÁLISIS DETALLADO POR CLÁUSULA


,Texto,Clasificación,Confianza
0,TRANSCRIPCIÓN COMPLETA:\r\n===================...,EVIDENCE,48.8%
1,de media en todo el planeta las temperaturas e...,CLAIM,45.5%
2,de media en todo el planeta\r\n[53. 76s - 56. ...,EVIDENCE,56.7%
3,tampoco es algo que los\r\n[96. 32s - 98. 76s]...,EVIDENCE,44.2%
4,el cambio climático actual no tiene precedente...,CLAIM,42.3%
5,a la vez más simplista para remeter\r\n[262,NONE,70.4%
6,08s]: contra la credibilidad de los científic...,NONE,82.9%
7,48s]: En 1988 las naciones unidas creó el gru...,EVIDENCE,69.8%
8,"32s]: climático, el IPCF",EVIDENCE,91.9%
9,44s]: Se trata de miles de expertos de más de...,EVIDENCE,81.9%



... y 13 cláusulas más

🎉 ¡Análisis completado!

📝 INSTRUCCIONES:
1. Pega tu transcripción en la celda marcada
2. Ejecuta todas las celdas (Runtime > Run all)
3. Revisa los resultados en las visualizaciones
4. Descarga los archivos HTML y JSON generados
